# Low Grade Glioma - 1p19q status

Import some useful libraries

In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2

/Users/blezek/Source/dfci/miniconda/lib/python2.7/site-packages/openpyxl/reader/worksheet.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
# Create an HD5 file containing all our data by using the function in lggData
%run 'lggData.ipynb'

lgg_create_dataset ( 'data.h5', 'NiFTiSegmentationsEdited', 'TCIA_LGG_cases_159.xlsx' )

NameError: global name 'h5py' is not defined

In [2]:
# Set our backend
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
import keras
import keras_resnet.models

input = keras.layers.Input ( (1,256, 256) )
model = keras_resnet.models.ResNet50(input, classes=1)

Using TensorFlow backend.


In [4]:
# Import the optimizer
from keras.optimizers import Adam
from keras.objectives import binary_crossentropy


model.compile(optimizer=Adam(lr=0.001), loss=binary_crossentropy, metrics=['binary_accuracy'])



In [5]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, TensorBoard

# Save our weights
weightsFilename = 'weights.h5'
tensorboardDirectory = 'logs'

# Tensorboard so we can monitor what's happening
modelCheckpoint = ModelCheckpoint( weightsFilename, verbose=1, monitor='loss',save_best_only=True)
earlyStoppingCallback = EarlyStopping(monitor='loss', patience=2)
callbacks = [modelCheckpoint,earlyStoppingCallback]
tensorboardCallback = TensorBoard(log_dir=tensorboardDirectory)
callbacks.append ( tensorboardCallback )

gen = lgg_status_generator ( 'NiFTiSegmentationsEdited', 'TCIA_LGG_cases_159.xlsx')

history = model.fit_generator ( 
    gen,
    steps_per_epoch=500,
    epochs=21,
    shuffle=True,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/21
 58/500 [==>...........................] - ETA: 20:23 - loss: 6.0815 - binary_accuracy: 0.6185skipping NiFTiSegmentationsEdited/LGG-326/LGG-326_T1c.nii.gz, data is wrong shape.  Expected 256x256, got (512, 512, 32)
skipping NiFTiSegmentationsEdited/LGG-223
 70/500 [===>..........................] - ETA: 19:55 - loss: 6.0638 - binary_accuracy: 0.6196

KeyboardInterrupt: 